In [1]:
#Quick fix for string labels in YOLO txt files accidentally imported from other formats

import os

# Define the mapping of class names to numeric labels
class_mapping = {
    #'Bird': 0,
    'Crane': 0,
    'Duck': 1,
    'Goose': 2,
    'Seagull': 3
}

# Path to the YOLO training labels directory
labels_directory = ''

# Iterate over the labels subdirectories
for root, dirs, files in os.walk(labels_directory):
    for file in files:
        if file.endswith('.txt'):
            file_path = os.path.join(root, file)
            with open(file_path, 'r') as f:
                lines = f.readlines()

            # Re-code the class labels
            new_lines = []
            for line in lines:
                class_name, *rest = line.strip().split(' ')
                if class_name in class_mapping:
                    class_label = class_mapping[class_name]
                    new_line = f"{class_label} {' '.join(rest)}\n"
                    new_lines.append(new_line)

            # Write the re-coded labels back to the file
            with open(file_path, 'w') as f:
                f.writelines(new_lines)


In [ ]:
#Quick script to fix prohibited YOLO label values >1 or <0

import os

# Path to the directory containing the labels
labels_directory = ''

# List to store filenames of changed image files
changed_image_files = []

# Iterate over the labels subdirectories
for root, dirs, files in os.walk(labels_directory):
    for file in files:
        if file.endswith('.txt'):
            file_path = os.path.join(root, file)
            image_file = os.path.splitext(file)[0] + '.jpg'  # Assuming the images have the same name but with a different extension

            with open(file_path, 'r') as f:
                lines = f.readlines()

            # Modify the YOLO bounding box coordinates in each line
            modified_lines = []
            for line in lines:
                values = line.strip().split(' ')
                class_label = values[0]
                modified_values = [class_label]

                for i in range(1, len(values)):
                    value = float(values[i])
                    if i % 2 == 1:  # X-coordinate or width
                        if value < 0:
                            value = 0
                        elif value > 1:
                            value = 1
                    else:  # Y-coordinate or height
                        if value < 0:
                            value = 0
                        elif value > 1:
                            value = 1
                    modified_values.append(str(value))

                modified_line = ' '.join(modified_values) + '\n'
                modified_lines.append(modified_line)

            # Write the modified lines back to the file
            with open(file_path, 'w') as f:
                f.writelines(modified_lines)

            changed_image_files.append(image_file)

# Write the list of changed image files to a separate file
changed_files_path = os.path.join(labels_directory, 'changed_image_files.txt')
with open(changed_files_path, 'w') as f:
    f.write('\n'.join(changed_image_files))

print(f"Total changed image files: {len(changed_image_files)}")
print(f"List of changed image files saved to: {changed_files_path}")

In [1]:
#Multiclass to single class
#IMPORTANT: this overwrites labels, before running, make sure you have a copy of the original values saved!

import os

def update_labels_directory(directory):
    # Iterate over all files and subdirectories in the given directory
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".txt"):
                # Process only text files with YOLO format
                file_path = os.path.join(root, file)
                update_yolo_labels(file_path)

def update_yolo_labels(file_path):
    # Read the contents of the YOLO format text file
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Update the class values to 0
    updated_lines = [line.replace(line.split()[0], '0') for line in lines]

    # Write the updated contents back to the file
    with open(file_path, 'w') as f:
        f.writelines(updated_lines)

# Provide the path to your labels directory
labels_directory = 'E:\\datasets\\usfws\\labels\\'

# Call the function to update the labels in the directory
update_labels_directory(labels_directory)
